In [34]:
import os
import pandas as pd
import pytz
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from collections import defaultdict
from django.apps import apps as django_apps
from django.db.models import F, Max, Case, When, Sum
from edc_constants.constants import POS, NEG
from flourish_reports.views.enrolment.enrollment_report_mixin import EnrolmentReportMixin
from tqdm import tqdm

In [35]:
report_mixin = EnrolmentReportMixin()

In [36]:
def convert_to_title_case(snake_case_string):
    title_case_string = snake_case_string.replace("_", " ").title()
    return title_case_string

In [37]:
# Enrolment cohort categorization
enrol_total = report_mixin.enrollment_report

enrol_summary_df = pd.DataFrame(enrol_total)

enrol_summary_df

,cohort_name,unexposed,exposed
0,Cohort A,34,116
1,Cohort B,77,118
2,Cohort C,34,46
3,Cohort A Sec,0,0
4,Cohort B Sec,1,348
5,Cohort C Sec,12,116


In [38]:
enrol_summary_df.to_excel("enrollment_summary_report.xlsx", index=False)

In [39]:
# Current cohort categorization
current_total = report_mixin.current_report

current_summary_df = pd.DataFrame(current_total)

current_summary_df

,cohort_name,unexposed,exposed
0,Cohort A,2,32
1,Cohort B,55,200
2,Cohort C,91,100
3,Cohort A Sec,0,0
4,Cohort B Sec,0,139
5,Cohort C Sec,0,270


In [40]:
current_summary_df.to_excel("current_summary_report.xlsx", index=False)

In [41]:
def convert_to_regular_dict(d):
    if isinstance(d, defaultdict):
        d = {k: convert_to_regular_dict(v) for k, v in d.items()}
    return d

In [42]:
# Re-classification counts
final_classification = []
sequence_dict = convert_to_regular_dict(report_mixin.get_sequence)
for enrol_cohort, movements in sequence_dict.items():
    for curr_cohort, total in movements.items():
        final_classification.append(
            {'enrollment_cohort': enrol_cohort, 'current_cohort': curr_cohort, 'count': total})

recategorization_df = pd.DataFrame(final_classification)
recategorization_df

,enrollment_cohort,current_cohort,count
0,cohort_b,cohort_c,79
1,cohort_b,cohort_c_sec,15
2,cohort_b_sec,cohort_b,60
3,cohort_b_sec,cohort_c,34
4,cohort_b_sec,cohort_c_sec,141
5,cohort_c_sec,cohort_c,12
6,cohort_a,cohort_b,94
7,cohort_a,cohort_b_sec,23


In [43]:
recategorization_df.to_excel("recategorization_report.xlsx", index=False)
recategorization_df

,enrollment_cohort,current_cohort,count
0,cohort_b,cohort_c,79
1,cohort_b,cohort_c_sec,15
2,cohort_b_sec,cohort_b,60
3,cohort_b_sec,cohort_c,34
4,cohort_b_sec,cohort_c_sec,141
5,cohort_c_sec,cohort_c,12
6,cohort_a,cohort_b,94
7,cohort_a,cohort_b_sec,23
